#
# Name                    Version                   Build  Channel
bzip2                     1.0.8                h2bbff1b_6
ca-certificates           2025.2.25            haa95532_0
expat                     2.7.1                h8ddb27b_0
libffi                    3.4.4                hd77b12b_1
openssl                   3.0.17               h35632f6_0
pip                       25.1               pyhc872135_2
python                    3.11.13              h981015d_0
setuptools                78.1.1          py311haa95532_0
sqlite                    3.50.2               hda9a48d_1
tk                        8.6.14               h5e9d12e_1
tzdata                    2025b                h04d1e81_0
ucrt                      10.0.22621.0         haa95532_0
vc                        14.3                h2df5915_10
vc14_runtime              14.44.35208         h4927774_10
vs2015_runtime            14.44.35208         ha6b5a95_10
wheel                     0.45.1          py311haa95532_0
xz                        5.6.4                h4754444_1
zlib                      1.2.13               h8cc25b3_1

In [27]:
# Aplikacja do nauki języka angielskiego z wykorzystaniem OpenAI
# import instructor  # import jest wymagany do generowania słówek i wskazówek
import streamlit as st # Wykorzystuje Streamlit do stworzenia prostego interfejsu,
from dotenv import dotenv_values # dotenv do zarządzania konfiguracją i przechowywania klucza API,
from openai import OpenAI # OpenAI do generowania pytań i odpowiedzi,
import speech_recognition as sr # do rozpoznawania mowy

v1 - możliwość dodawania wiadomości w języku polskim i tłumaczenia na język angielski

In [28]:
# Wczytanie zmiennych środowiskowych
def load_environment():
    """Ładuje zmienne środowiskowe z pliku .env"""
    env = dotenv_values(".env")
    if not env.get("OPENAI_API_KEY"):
        st.error("❌ Brak klucza API OpenAI w pliku .env")
        st.stop()
    return env

env = load_environment()  # <-- dodaj to wywołanie funkcji
client = OpenAI(api_key=env.get("OPENAI_API_KEY"))


In [29]:
# Lista obsługiwanych języków ( można rozszerzyć)
supported_languages = ["angielski", "polski", "niemiecki", "francuski", "hiszpański", "włoski"]

st.title("Jack - osobisty asystent nauki języków")

with st.sidebar:
    tool_language = st.selectbox(
        "Wybierz narzędzie",
        [
            "Translator", # tłumacz ZROBIONE
            "Belfer", # sprawdza poprawność zdań ZROBIONE
            "Jak powiem?", # pomaga budować zagadnienia
            "Dialog" # prowadzi dialog (dorobić wybór tematów.)
        ], key="tool_language"
    )

2025-09-05 14:33:02.075 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.076 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.078 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.080 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.081 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.083 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:33:02.085 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [ ]:
na razie nieużywana

# Funkcja do rozpoznawania mowy za pomocą Whisper API
def recognize_speech_whisper(audio_file):
    with open(audio_file, "rb") as file:
        transcription = client.audio.transcriptions.create(
            model="whisper-1",
            file=file,
            language="pl"  # Kod języka, np. "en" dla angielskiego
        )
    return transcription.text

In [30]:
# Funkcja do rozpoznawania mowy za pomocą biblioteki SpeechRecognition i mikrofonu (potrzebny mikrofon) 
# ustawienie języka kodu dla rozpoznawania mowy
language_code_map = {
    "angielski": "en-US",
    "polski": "pl-PL",
    "niemiecki": "de-DE",
    "francuski": "fr-FR",
    "hiszpański": "es-ES",
    "włoski": "it-IT"
}       
language_code = language_code_map.get(language_in, "pl-PL")  # Domyślnie polski   
def recognize_speech(language=language_code):
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        st.write("🎤 Mów teraz...")
        audio = recognizer.listen(source)
        try:
            text = recognizer.recognize_google(audio, language=language)
            return text
        except sr.UnknownValueError:
            return "Nie udało się rozpoznać mowy."
        except sr.RequestError as e:
            return f"Błąd podczas rozpoznawania: {e}"

In [37]:
# Translator
if tool_language == "Translator":
    st.header("Translator językowy")

     # wybór języka wprowadzonego tekstu
    language_in= st.selectbox("Wybierz język tekstu do tłumaczenia:", supported_languages, key="language_in")


    # Pole tekstowe do wpisania wiadomości
    translate_text = st.text_area("Wpisz tekst do tłumaczenia w obsługiwanym języku:")

    # wybór języka tłumaczenia
    language_out= st.selectbox("Wybierz język tłumaczenia:", supported_languages, key="language_out")

  

    
    if st.button("Przetłumacz na wybrany język"):
        if translate_text is None or (hasattr(translate_text, "strip") and not translate_text.strip()) or translate_text == "Nie udało się rozpoznać mowy." or translate_text.startswith("Błąd podczas rozpoznawania:"):
            st.warning("Proszę wpisać tekst do przetłumaczenia lub nagrać rozmowę.")
            
        else:
            # Wywołanie OpenAI API do tłumaczenia
            prompt = f"Przetłumacz na {language_out} następujący tekst:\n{translate_text}"
            try:
                response = client.chat.completions.create(
                    model="gpt-4o-mini-transcribe",
                    messages=[
                        {"role": "system", "content": f"Jesteś pomocnym tłumaczem zjęzyka {language_in} na język {language_out}. Jeśli tekst jest już w wybranym języku {language_out}, odpowiedz 'Tekst jest już w wybranym języku.', Jeśli tekst jest w innym języku niż {supported_languages}, odpowiedz 'Język podanego tekstu nie jest obsługiwany.' "},
                        {"role": "user", "content": prompt}
                    ],
                    max_tokens=500,
                    temperature=0.1,
                )
                content = response.choices[0].message.content
                translation = content.strip() if content is not None else ""
                st.subheader(f"Tłumaczenie na {language_out}:")
                st.write(translation)
            except Exception as e:
                st.error(f"Wystąpił błąd podczas tłumaczenia: {e}")

    


2025-09-05 14:35:04.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.700 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.701 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.703 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.704 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.706 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.707 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:35:04.708 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar

In [36]:
# Sekcja rozpoznawania mowy
    #if st.button("Nagraj mowę"):
translate_text = recognize_speech(language=language_code)  # Zmień kod języka w zależności od wyboru
# st.text_area("Rozpoznany tekst:", value=recognized_text, key="recognized_text")
print(translate_text)

2025-09-05 14:34:45.057 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:34:45.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-05 14:34:45.058 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


directions to


In [20]:
print(language_in)


angielski
